In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

model_names = ['original_model','trained_model']
data_path = '../data/pose_model/model_predictions_and_groundtruth.pkl'

df = pd.read_pickle(data_path)

for i in model_names:
    df[i, 'x_square_error'] = ((df[i].x - df.ground_truth.x)/df.ground_truth.bbox_width)**2 
    df[i, 'y_square_error'] =  ((df[i].y-df.ground_truth.y)/df.ground_truth.bbox_height)**2

df = df.sort_index(axis = 1).set_index(['video', 'frame', 'id', 'part_label'])
df = df.reset_index()
df = df[~df['part_label', ''].isnull()]

group_by = ['video', 'frame']

def compute_metrics(df, model_names):
    precision_list = []; recall_list = []; rmse_list = [];
    relevant_elements = df.loc[~df.ground_truth.x.isnull(),:]
    for i, imodel in enumerate(model_names):
        recall =  np.sum(~relevant_elements[imodel].x.isnull())/len(relevant_elements)
        selected_items = df.loc[~df[imodel].x.isnull(),:]
        precision = np.sum(~selected_items.ground_truth.x.isnull())/(len(selected_items)-1) # -1 to leave out neck kp
        precision_list.append(precision); recall_list.append(recall)
        rmse = np.sqrt(np.nanmean(df[imodel].x_square_error.tolist() + df[imodel].y_square_error.tolist()))
        rmse_list.append(rmse)
    return pd.DataFrame.from_dict({'model': model_names, 'precision': precision_list, 'recall': recall_list, 'rmse': rmse_list})  

performance_metric_df = df.groupby(group_by).apply( lambda x: compute_metrics(x, model_names))

In [ ]:
zdf = performance_metric_df
zdf = pd.pivot_table(zdf, values=['precision','recall', 'rmse'],\
                index=group_by, columns = 'model').reset_index()

col = ['_'.join(i).strip() for i in zdf.columns.values]
zdf = zdf.dropna()
zdf.columns = col
zdf['video'] = 'video_'+zdf['video_'].astype(str)

labelzr1 = ['A','C', 'E']
labelzr2 = ['B','D', 'F']

fontsize = 13
fontsize_panel = 25

zdf['recall_d'] = zdf['recall_trained_model'] - zdf['recall_original_model']
zdf['precision_d'] = zdf['precision_trained_model'] - zdf['precision_original_model']
zdf['rmse_d'] = zdf['rmse_trained_model'] - zdf['rmse_original_model']

xs = ['rmse_original_model', 'precision_original_model', 'recall_original_model']
ys = ['rmse_trained_model', 'precision_trained_model', 'recall_trained_model']
xlabels = ['RMSE (OpenPose model)', 'Precision (OpenPose model)','Recall (OpenPose model)']
ylabels = ['RMSE (Trained model)', 'Precision (Trained model)','Recall (Trained model)']
lims = [1,1.02,1.02]
fig,axs=plt.subplots(nrows=2,ncols=3, figsize=(15,10))

for i in np.arange(3):
    ax = axs[0,i]
    ax.text(-.1, 1.1, labelzr1[i], transform=ax.transAxes,fontsize=fontsize_panel, va='bottom', ha='right')
    sns.scatterplot(data=zdf, x=xs[i],\
                    edgecolor='none',y=ys[i], alpha=.5, ax=ax)
    ax.plot([0 ,lims[i]], [0,lims[i]], color=[0,0,0], ls='--', alpha=.5)
    ax.set_xlabel(xlabels[i], fontsize=fontsize)
    ax.set_ylabel(ylabels[i],fontsize = fontsize)
    ax.set_xticks(np.arange(0,1.1,.2))
    ax.set_xticklabels(np.around(np.arange(0,1.1,.2),decimals=2),fontsize=fontsize)
    ax.set_yticks(np.arange(0,1.1,.2))
    ax.set_yticklabels(np.around(np.arange(0,1.1,.2),decimals=2),fontsize=fontsize)
    ax.set_xlim(0, lims[i])
    ax.set_ylim(0, lims[i])

plot_var = ['rmse_d', 'precision_d','recall_d']
summary_var = ['rmse','precision','recall']
ylims = [1500,2500,1500]
xlabels = ['RMSE (Trained) - RMSE (OpenPose)', 'Precision (Trained) - Precision (OpenPose)','Recall (Trained) - Recall (OpenPose)']
xlims_l = [-.2,-.4,-1]
xlims_u = [.2,.4,1]
xsp = [.1,.2,.5]
ysp = [500,500,500]
s = [int(10/i) for i in xsp]


for i,ivar in enumerate(plot_var):
    ax = axs[1,i]
    ax.text(-.1, 1.1, labelzr2[i], transform=ax.transAxes,fontsize=fontsize_panel, va='bottom', ha='right')
    sns.distplot(zdf[ivar], bins=s[i], ax=ax, kde=False, hist=True, norm_hist=False)
    
    second_ax = ax.twinx()
    sns.distplot(zdf[ivar], bins=s[i], ax=second_ax, kde=True, hist=False)
    second_ax.set_yticks([])
    ax.plot([0,0],[0,ylims[i]], color=[0,0,0], ls='-')
    ax.plot([metrics[summary_var[i]][1] - metrics[summary_var[i]][0]]*2,[0,ylims[i]], color=[1,0,0], alpha=.7, ls='--')
    ax.set_xlabel(xlabels[i],fontsize=fontsize)
    ax.set_ylabel('# Images',fontsize=fontsize)
    ax.set_xlim(xlims_l[i],xlims_u[i])
    ax.set_ylim(0,ylims[i])
    ax.set_yticks(np.arange(0,ylims[i]+1,ysp[i]))
    ax.set_yticklabels(np.arange(0,ylims[i]+1,ysp[i]),fontsize=fontsize)
    ax.set_xticks(np.around(np.arange(xlims_l[i],xlims_u[i]+.1,xsp[i]),decimals=2))
    ax.set_xticklabels(np.around(np.arange(xlims_l[i],xlims_u[i]+.2,xsp[i]),decimals=2),fontsize=fontsize)

sns.despine()
fig.subplots_adjust(left=.1, bottom=.1, right=.9, top=.9, wspace=.4, hspace=.4)

fig.savefig('../data/visualization/training_plot.pdf') 

metrics = compute_metrics(df, model_names)